Links:
1. https://stackoverflow.com/questions/58718764/how-to-create-a-color-bar-using-a-dictionary-in-python
2. https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.broken_barh.html

In [1]:
import cfgrib
from datetime import datetime, timedelta
from ecmwf.opendata import Client
from ipyleaflet import Choropleth, Map, basemap_to_tiles, LayersControl, LegendControl, ImageOverlay
import ipywidgets as widgets
from IPython.display import display
from localtileserver import get_leaflet_tile_layer, TileClient
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import rasterio
import rioxarray as rxr
import xarray as xr

from ipyleaflet import ColormapControl
import branca.colormap as bc

In [2]:
def get_colordict(array, cmap, n_col = 8):
    """
    array: numpy.array
        Array containing the values of which the quantiles will be obtained
    cmap: str
        Name of the matplotlib colormap associated to the array

    Returns:
    dict
        Dictionary containing Hex codes as keys and the array quantiles as values
    """
    cmap = plt.get_cmap(cmap, n_col)
    custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]
    quantiles = [round(np.quantile(array, q), 1) for q in np.linspace(0,1,n_col)]
    return(dict(zip(quantiles, custom_palette)))

def get_colorbar(colordict, title, var):
    """
    Create a plot with the colormap to be added to the ipyleaflet map

    colordict: dict
        Output of get_colordict
    title: str
        Title to be assigned to the colormap
    var: str
        Name of the variable for which the colorbar is being computed. Needed to
        create the output path
    
    Returns:
    imgpath: str
        Path to the colorbar plot, saved as .png
    """
    squares = [(i, 1) for i in range(len(colordict.keys()))]
    colors = [colordict[i] for i in colordict.keys()]
    fig, ax = plt.subplots(1,1, )
    plt.broken_barh(squares, (0,1), facecolors = colors)
    # set x labels
    ax.set_xticks([x+0.5 for x in range(len(colordict.keys()))])
    ax.set_xticklabels(colordict.keys())
    ax.tick_params('x', length = 0)
    for tick in ax.get_xticklabels():
        tick.set_fontname("tahoma")
    # remove bounding box and y ticks
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.get_yaxis().set_ticks([])
    # set aspect ration
    ax.set_aspect(0.5)
    # set title
    ax.set_title(title, fontname = "tahoma")
    # save and return path
    imgpath = f"data/colorbars/colorbar_{var}.png"
    fig.savefig(imgpath, bbox_inches = "tight")
    plt.close(fig)
    return(imgpath)

In [26]:
r = rasterio.open("C:/repos/TropiDash/notebooks-examples-trials/data/atm/tp_20230903_time0_step240_oper_fc.tiff")
coldict = get_colordict(r.read(1).ravel(), "Spectral", 10)
imgpath = get_colorbar(coldict, "test2", "test2")

In [12]:
#look for standard palettes for atmospherical values

In [17]:
# create a function which, for a given palette, creates a branca palette

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [8]:
test = plt.get_cmap("viridis", 10)

In [11]:
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])

In [5]:
tempp = [
(254, 254, 203),
(251, 235, 153),
(244, 204, 104),
(235, 167, 84),
(228, 134, 80),
(209, 98, 76),
(164, 70, 66),
(114, 55, 46),
(66, 40, 24),
(25, 25, 0)
]

In [13]:
tempphex = [rgb_to_hex(x) for x in tempp]

In [19]:
bc.StepColormap(tempphex)

In [ ]:
brancacmap

In [15]:
palette = ["#c4ff70", "#6ae24c", "#2a9134", "#137547", "#054a29", "#2397d1", "#557ff3", "#143cdc", "#3910b4", "#1e0063"]
def make_palette_dict(pal):
    perc = ["0-10%", "11-20%", "21-30%", "31-40%", "41-50%", "51-60%", "61-70%", "71-80%", "81-90%", "91-100%"]
    dictionary = {perc[0]:pal[0], perc[1]:pal[1], perc[2]:pal[2], perc[3]:pal[3], perc[4]:pal[4], perc[5]:pal[5], perc[6]:pal[6], perc[7]:pal[7], perc[8]:pal[8], perc[9]:pal[9]}
    return dictionary

In [29]:
m = Map(zoom = 3)
client = TileClient(r)
t = get_leaflet_tile_layer(client, name = "Mean sea level pressure [kPa]", opacity = 0.7, palette = prechex)
m.add_layer(t)
# add colorbar
minv = "%.2f" % round(r.read(1).ravel().min(), 1)
maxv = "%.2f" % round(r.read(1).ravel().max(), 1)
colormap_control = ColormapControl(caption = "Mean sea level pressure [kPa]",
                                   colormap = bc.StepColormap(prechex),
                                   value_min = minv,
                                   value_max = maxv,
                                   position='topright',
                                   transparent_bg=True
                                   )
m.add(colormap_control)
m.add(LayersControl())
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [23]:
prec = [(255, 255, 229),(217, 235, 213),(180, 216, 197),(142, 197, 181),(105, 177, 165),(67, 158, 149),(44, 135, 127),(29, 110, 100),(14, 85, 74),(0, 60, 48)]

In [25]:
prechex = [rgb_to_hex(x) for x in prec]
bc.StepColormap(prechex)